# Classification and Regression Tree (CART)

## Titanic Dataset
- Kaggle knowledge competition: https://www.kaggle.com/c/titanic

In [1]:
# read in the data
import pandas as pd
url = 'data/titanic.csv'
titanic = pd.read_csv(url)

# encode female as 0 and male as 1
titanic['Sex'] = titanic.Sex.map({'female':0, 'male':1})

# fill in the missing values for age with the median age
titanic.Age.fillna(titanic.Age.median(), inplace=True)

# create a DataFrame of dummy variables for Embarked
embarked_dummies = pd.get_dummies(titanic.Embarked, prefix='Embarked')
embarked_dummies.drop(embarked_dummies.columns[0], axis=1, inplace=True)

# concatenate the original DataFrame and the dummy DataFrame
titanic = pd.concat([titanic, embarked_dummies], axis=1)

# print the updated DataFrame
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Embarked_Q,Embarked_S
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,S,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C,0,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S,0,1
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,S,0,1


In [2]:
# define X and y
feature_cols = ['Pclass', 'Sex', 'Age', 'Embarked_Q', 'Embarked_S']
X = titanic[feature_cols]
y = titanic.Survived

In [3]:
# fit a classification tree with max_depth=3 on all data
from sklearn.tree import DecisionTreeClassifier
treeclf = DecisionTreeClassifier(max_leaf_nodes=10)
treeclf.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=10,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [4]:
treeclf.decision_path(X)

<891x19 sparse matrix of type '<class 'numpy.int64'>'
	with 3658 stored elements in Compressed Sparse Row format>

## CART 그림 도출 - 1 (추천 방법)

- `sklearn.tree.export_graphviz`를 이용하여 **.dot** 파일을 생성
- 생성된 dot 파일을 텍스트 편집기에서 불러들인 후, 이를 복사하여 [WebGraphviz.com](http://webgraphviz.com/)에 붙여넣으면 의사결정나무 그림을 얻을 수 있음

In [4]:
# Alternatively, you can create tree images
# using dot file and WebGraphviz (http://webgraphviz.com/)
from sklearn.tree import export_graphviz

with open("titanic_tree.dot", "w") as f:
    f = export_graphviz(treeclf, 
                        out_file = f, 
                        feature_names = X.columns,
                        class_names = ["Dead", "Survived"], # 클래스가 [0,1]이므로 이 순서를 맞춰서 이름을 할당
                        filled = True,
                        rounded = True)

## CART 그림 도출 - 2

- **GraphViz** 프로그램 다운로드 후 설치: http://www.graphviz.org/
- 환경변수(PATH)에 GraphViz 실행파일 경로를 추가: ex) 'C:\Program Files (x86)\Graphviz2.38\bin'를 환경변수에 추가
- **pydotplus** 패키지 설치: pip install pydotplus

In [ ]:
# !pip install pydotplus

In [ ]:
# create a Graphviz file and make pdf or png files
# You must install GraphViz
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO
import pydotplus
from IPython.display import Image 
dot_data = StringIO()
export_graphviz(treeclf, out_file= dot_data)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
# graph.write_pdf("titanic_tree.pdf")
# graph.write_png("titanic_tree.png")

## Hands-on

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                   random_state=123)

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                  test_size=0.375,
                                                  random_state=123)

In [9]:
from sklearn.metrics import accuracy_score

max_depth_set = [3, 4, 5, 6, 7, 8]

for max_depth in max_depth_set:
    dt = DecisionTreeClassifier(max_depth=max_depth)
    dt.fit(X_train, y_train)
    y_val_pred = dt.predict(X_val)
    print(accuracy_score(y_val, y_val_pred))

0.8052434456928839
0.8277153558052435
0.8202247191011236
0.7940074906367042
0.7415730337078652
0.7715355805243446


## Homework
1. k-NN과 Logistic Regression도 여러 가지 세팅으로 Training and Validation
2. Test set에 베스트 모델을 평가